<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [3]:
#!mamba create -n mamba_env python=3.9
#!mamba activate mamba_env
!mamba install pandas -y
!mamba install requests -y
!mamba install curl -y


Looking for: ['pandas']


Pinned packages:
  - python 3.10.*


Transaction

  Prefix: C:\Users\admin\mambaforge

  All requested packages already installed


Looking for: ['requests']

conda-forge/win-64                                          Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.10.*


Transaction

  Prefix: C:\Users\admin\mambaforge

  All requested packages already installed


Looking for: ['curl']

conda-forge/win-64                                          Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.10.*


Transaction

  Prefix: C:\Users\admin\mambaforge

  All requested packages already installed



## Imports

Import any additional libraries you may need here.


In [4]:
import pandas as pd
from datetime import datetime
import json 
from typing import List 

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section


In [5]:
#!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
#!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
#!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv


import subprocess

#subprocess.run(['mamba', 'install', 'curl'])  # Install curl if needed

subprocess.run(['curl', '-o', 'bank_market_cap_1.json', 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json'])
subprocess.run(['curl', '-o', 'bank_market_cap_2.json', 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json'])
subprocess.run(['curl', '-o', 'exchange_rates.csv', 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv'])



CompletedProcess(args=['curl', '-o', 'exchange_rates.csv', 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv'], returncode=0)

## Extract


### JSON Extract Function

This function will extract JSON files.


In [37]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=False)
    #print(dataframe)
    return dataframe


## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [73]:
def extract():
    with open('bank_market_cap_1.json') as f:
        data = json.load(f)
    
    # Convert dictionary of dictionaries to list of dictionaries
    banks = []
    for i in range(len(data["Name"])):
        bank = {}
        for key in data:
            bank[key] = data[key][str(i)]
        banks.append(bank)
    
    # Create DataFrame from list of dictionaries
    df = pd.DataFrame(banks, columns=['Name', 'Market Cap (US$ Billion)'])
    print(df)
    return df

print(extract())


                                       Name Market Cap (US$ Billion)
0                            JPMorgan Chase                  390.934
1   Industrial and Commercial Bank of China                  345.214
2                           Bank of America                  325.331
3                               Wells Fargo                  308.013
4                   China Construction Bank                  257.399
..                                      ...                      ...
65                             Ping An Bank                   37.993
66                       Standard Chartered                   37.319
67                     United Overseas Bank                   35.128
68                                QNB Group                   33.560
69                              Bank Rakyat                   33.081

[70 rows x 2 columns]
                                       Name Market Cap (US$ Billion)
0                            JPMorgan Chase                  390.934
1   Industr

<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [25]:
exchange_rates = pd.read_csv('exchange_rates.csv', index_col=0)
df_exchange_rates = pd.DataFrame(exchange_rates)
symbol = 'GBP'
exchange_rate = df_exchange_rates.loc[symbol]
print(symbol, 'exchange rate with USD is: ',exchange_rate, exchange_rates[:5])  #GBP exchange rate

GBP exchange rate with USD is:  Rates    0.732398
Name: GBP, dtype: float64         Rates
AUD  1.297088
BGN  1.608653
BRL  5.409196
CAD  1.271426
CHF  0.886083


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [85]:

def transform(df, exchange_rate):
    # Convert Market Cap from USD to GBP
    usMarCap = 'Market Cap (US$ Billion)'
    gbpMarCap = 'Market Cap (GBP$ Billion)'
    df[usMarCap] = df[usMarCap].astype(float)  # Convert string values to float
    df[gbpMarCap] = df[usMarCap] * exchange_rate
    
    # Rename column
    #df = df.rename(columns={
     #   'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'
    #})
    
    # Drop the old USD column
    df = df.drop('Market Cap (US$ Billion)', axis=1)
    
    # Set index to name column
    df = df.set_index('Name')
    
    return df


    
#call the function
transform(extract(), exchange_rate)


                                       Name Market Cap (US$ Billion)
0                            JPMorgan Chase                  390.934
1   Industrial and Commercial Bank of China                  345.214
2                           Bank of America                  325.331
3                               Wells Fargo                  308.013
4                   China Construction Bank                  257.399
..                                      ...                      ...
65                             Ping An Bank                   37.993
66                       Standard Chartered                   37.319
67                     United Overseas Bank                   35.128
68                                QNB Group                   33.560
69                              Bank Rakyat                   33.081

[70 rows x 2 columns]


,Market Cap (GBP$ Billion)
Name,
JPMorgan Chase,NaN
Industrial and Commercial Bank of China,NaN
Bank of America,NaN
Wells Fargo,NaN
China Construction Bank,NaN
...,...
Ping An Bank,NaN
Standard Chartered,NaN
United Overseas Bank,NaN


## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [ ]:
targetfile = 'bank_market_cap.gbp.csv' #file for transformed data

def load(targetfile, df_exchange_rates):
    df_exchange_rates.to_csv(targetfile, index=False)    

## Logging Function


Write the logging function <code>log</code> to log your data:


In [ ]:
logfile = 'logfile.txt'  #store event logs

def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [ ]:
log("ETL Job Started")
log("Extract phase Started")

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [47]:
    # Call the function 
extracted_data = extract() 
    # Print the rows 
print(extracted_data)


                                                Name  \
0  {'0': 'JPMorgan Chase', '1': 'Industrial and C...   

                            Market Cap (US$ Billion)  
0  {'0': '390.934', '1': '345.214', '2': '325.331...  
None


Log the data as <code>"Extract phase Ended"</code>


In [ ]:
log("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [ ]:
# Write your code here
log("Transform phase Started")



<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [50]:
# Call the function here
int_currency = 1  #USD
transformed_data = transform(extracted_data, exchange_rate)
# Print the first 5 rows here
print(transformed_data)

TypeError: 'NoneType' object is not subscriptable

Log your data <code>"Transform phase Ended"</code>


In [ ]:
# Write your code here
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [ ]:
# Write your code here
log("Load phase Started")

Call the load function


In [ ]:
# Write your code here

load(targetfile,transformed_data)
    

NameError: name 'transformed_data' is not defined

Log the following `"Load phase Ended"`.


In [ ]:
# Write your code here
log("Load phase Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
